# Training data creation

In [6]:
#pip install autograd

In [7]:
import sys
sys.path.append("../modules")
import numpy as np
import pandas as pd
from numpy.linalg import inv, norm
from astropy import units as u
from datetime import datetime
from numba import jit
from scipy.optimize import minimize, fsolve, newton
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.plotting import *
from poliastro.constants import J2000
from poliastro.util import time_range
from datetime import datetime, timedelta
import plotly.io as pio
pio.renderers.default = "vscode"
from orbit_module import get_orbit, tle_data, create_rso, rv_sample
from observation import rv_observation, collision_model, objective, mass_constraint1, \
    mass_constraint2, velocity_constraint, partial_Lagrangian, hessian_lagrangian, svdinv, lagrangian
from astropy.constants import G, M_earth
import matplotlib.pyplot as plt
from astropy import time
from astropy.coordinates import EarthLocation, AltAz
import astropy.coordinates as coord
from tqdm import tqdm

mu = G*M_earth

In [8]:
sat_Data = pd.read_csv('../data/sat_list.csv', header = None)
sat_list = sat_Data.loc[:,0] + "-2-" + sat_Data.loc[:,2]

In [9]:
sat_list

0              LEMUR-2-JOEL
1             LEMUR-2-CHRIS
2            LEMUR-2-JEROEN
3             LEMUR-2-PETER
4     LEMUR-2-THERESACONDOR
              ...          
61     LEMUR-2-TALLHAMN-ATC
62         LEMUR-2-JIN-LUEN
63      LEMUR-2-URAMCHANSOL
64             LEMUR-2-KADI
65      LEMUR-2-THENICKMOLO
Length: 66, dtype: object

In [10]:
epoch = time.Time('2021-06-7 12:00:00', format='iso', scale='utc')
sat_no = 20

m_rso = (0.5 + 0.2*np.random.randn(sat_no))*u.kg
m_sat = (2 + 0.5*np.random.randn(sat_no))*u.kg
Z = np.zeros([sat_no, 6])
rso_data = np.zeros([sat_no, 7])
frame = OrbitPlotter3D()
for i in tqdm(range(0, sat_no)):
    active_sat_tle = get_orbit(sat_list[i])
    delt = epoch - active_sat_tle.epoch
    active_sat = active_sat_tle.propagate(delt)
    rso_bc = create_rso(active_sat, 0.6*np.random.randn(3)*u.km/u.s)
    collision = collision_model(m_sat[i], m_rso[i], active_sat, rso_bc)

    r_z = collision.sat.r.value + 0.5*np.random.randn(3)
    delv_z = collision.delta_v_sat.value + 0.1*np.random.randn(3)
    Z[i, :] = np.hstack((r_z, delv_z))
    rso_data[i, :] = np.hstack((rso_bc.r.value, rso_bc.v.value, m_rso[i].value))
    #if i%3 == 0:
    frame.plot(active_sat, label = 'sat' + str(i))
    

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]


TypeError: get_orbit() missing 1 required positional argument: 'TLE'

In [38]:
training_Data = pd.DataFrame(np.hstack((Z, rso_data)), columns=['x', 'y', 'z', 'delvx', 'delvy', 'delvz', \
    'rso_x', 'rso_y', 'rso_z', 'rso_vx', 'rso_vy', 'rso_vz', 'rso_m'])
training_Data.to_csv('../data/training_data.csv', index = False)

In [39]:
len(sat_list)

66